In [1]:
import pandas as pd
import numpy as np
import pickle

In [207]:
# Training tweets as arrays
with open("train/data/tweet_by_ID_02_12_2019__10_46_39.txt.text") as tr_twt:
    train_tweets = tr_twt.readlines()
# training labels
with open("train/data/tweet_by_ID_02_12_2019__10_46_39.txt.labels", "r") as tr_labels:
    train_labels = tr_labels.readlines()
with open("test/us_test.text") as t:
    test_tweets = t.readlines()
# test labels
with open("test/us_test.labels", "r") as tl:
    test_labels = tl.readlines()

In [50]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vect = vectorizer.fit_transform(train_tweets)

In [72]:
print(vect.shape)
a = vectorizer.vocabulary_
test = train_tweets[0]
tens = []
for word in test.split():
    try:
        tens.append([a[word.lower()]])
    except:
        tens.append([a['unk']])
tens = [x[0] for x in tens]
print(tens)
test = torch.tensor(tens, dtype=torch.long)
print(test)

(437161, 275935)
[145742, 255376, 264473, 255376, 42139]
tensor([145742, 255376, 264473, 255376,  42139])


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [13]:
def Eval_LSTM(X, Y, lstm):
    with open("predictions.txt", "w") as f:
        num_correct = 0
        for i in range(len(X)):
            logProbs = lstm.forward(X[i], train=False)
            pred = torch.argmax(logProbs)
            f.write(str(pred.item()))
            f.write("\n")
            if pred == int(Y[i]):
                num_correct += 1
        print("Accuracy: %s" % (float(num_correct) / float(len(X))))

In [196]:
class LSTM(nn.Module):
    def __init__(self, X, Y,NUM_CLASSES=20):
        super(LSTM, self).__init__()
        self.Embedding = nn.Embedding(275935, 10)
        self.lstm = nn.LSTM(10,20)
        self.Relu = nn.ReLU()
        self.hidden = (torch.randn(1, 1, 20),
                  torch.randn(1, 1, 20))
        
        nn.init.xavier_uniform_(self.Embedding.weight)

    def forward(self, X, vocab, train=False):
        #TODO: Implement forward computation
        tens = []
        sen_len = len(X.split())
        for word in X.split():
            try:
                tens.append([a[word.lower()]])
            except:
                tens.append([a['unk']])
        tens = [x[0] for x in tens]
        X = torch.tensor(tens, dtype=torch.long)
        #X = X.view(1,1,-1).long()
        X = self.Relu(torch.sum(self.Embedding(X), dim = 0))
        X = X.unsqueeze(0)
        X = X.unsqueeze(0)
        out, self.hidden = self.lstm(X, self.hidden)
        return out

In [203]:
def Train_LSTM(X, vocab, Y, n_iter):
    print("Start Training!")
    lstm = LSTM(X,vocab, Y)
    #TODO: initialize optimizer.
    #optimizer = optim.SGD(mlp.parameters(), lr=.01)
    optimizer = optim.Adam(lstm.parameters(), lr=1e-4)
    Loss = torch.nn.CrossEntropyLoss()

    for epoch in range(n_iter):
        total_loss = 0.0
        for i in range(len(X)):
            if i % 100000 == 0:
                print(i)
            #TODO: compute gradients, do parameter update, compute loss.
            lstm.zero_grad()
            probs = lstm.forward(X[i], vocab)
            probs = probs.squeeze(0)
            y = float(Y[i])
            loss = Loss(probs,torch.Tensor([y]).long())
            total_loss += loss
            loss.backward(retain_graph=True)
            lstm.hidden[0].detach_()
            lstm.hidden[1].detach_()
            optimizer.step()
            
        print(f"loss on epoch {epoch} = {total_loss}")
    return lstm

In [ ]:
#train_tensor = torch.from_numpy(np.array(train_tweets))
lstm = Train_LSTM(train_tweets, vectorizer.vocabulary_, train_labels, 5)

Start Training!
0
100000
200000
300000
400000
loss on epoch 0 = 1178514.375
0
100000
200000
300000
400000
loss on epoch 1 = 1158411.375
0
100000
200000
300000
400000
loss on epoch 2 = 1150305.125
0
100000
200000
300000
400000
loss on epoch 3 = 1145241.75
0
100000


In [208]:
#X_test = torch.from_numpy(test_twts)
Eval_LSTM(test_tweets, test_labels, lstm)

NameError: name 'lstm' is not defined

In [ ]:
loss on epoch 0 = 1159363.375
loss on epoch 1 = 1121209.25
loss on epoch 2 = 1109531.375
loss on epoch 3 = 1103173.875
loss on epoch 4 = 1098848.625
loss on epoch 5 = 1095612.875
loss on epoch 6 = 1092917.75
loss on epoch 7 = 1090395.125
loss on epoch 8 = 1088438.875
loss on epoch 9 = 1086809.125
loss on epoch 10 = 1085420.75
loss on epoch 11 = 1084237.125
loss on epoch 12 = 1083184.75
loss on epoch 13 = 1082234.0
loss on epoch 14 = 1081408.375
loss on epoch 15 = 1080632.125
loss on epoch 16 = 1079940.5
loss on epoch 17 = 1079288.0
loss on epoch 18 = 1078705.25
loss on epoch 19 = 1078166.75
loss on epoch 20 = 1077661.75
loss on epoch 21 = 1077190.125
loss on epoch 22 = 1076737.75
loss on epoch 23 = 1076340.625
loss on epoch 24 = 1075950.75